# import 

In [ ]:
# 구글드라이브에서 파일 읽기
from google.colab import drive

# 데이터 프레임 및 연속형 자료 연산 라이브러리
import pandas as pd
import numpy as np

#시각화 라이븐러리
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

#통계수치 계산 라이브러리
from scipy import stats
from scipy.stats import variation

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,r2_score

# 쿠킹데이터 불러오기기

In [ ]:

drive.mount('/content/drive')
cooking=pd.read_csv('/content/drive/MyDrive/파이썬 2차프로젝트/Cooking_data_set.csv')
cooking

# 생산시간 0인값 지우기기

In [ ]:
data1=cooking.dropna(subset=['생산시간'])
data1

In [ ]:
data2=data1.sort_values('품목명')
data2

# 데이터 그룹화

In [ ]:
data1.columns

In [ ]:
data1['품목명'].unique()

In [ ]:
group1=data1.groupby('품목명', as_index=False)[ '충전실온도', '실링온도', '쿠킹온도', '쿠킹스팀압력', '실링압력', '생산시간']

In [ ]:
group1.groups
# 품목별로 최소,최대,평균과 같은 
# group1에 group1.agg(['size', 'mean', 'std', 'min', 'max']) 통계량을 입력해서 값들을 확인해라.

In [ ]:
group1.agg(['mean', 'min', 'max']) # 중간값, 

In [ ]:
group1.describe()

# 모델링

## 데이터 정리

In [ ]:
data1.columns

In [ ]:
data2

In [ ]:
data3=data2.drop(['순번', '생산라인코드', '생산라인명', '생산일자', '품목코드', '품목명', '작업장코드'],axis=1)
data3

# data2에서 사용하지 않는 칼럼 제거

In [ ]:
data_set=data3.dropna()

In [ ]:
data_set.isnull().sum()

In [ ]:
data_set.dtypes

In [ ]:
data_set['충전실온도'] = data_set['충전실온도'].astype({'충전실온도' : 'int'})
data_set['실링온도'] = data_set['실링온도'].astype({'실링온도' : 'int'})
data_set['쿠킹온도'] = data_set['쿠킹온도'].astype({'쿠킹온도' : 'int'})
data_set['쿠킹스팀압력'] = data_set['쿠킹스팀압력'].astype({'쿠킹스팀압력' : 'int'})
data_set['실링압력'] = data_set['실링압력'].astype({'실링압력' : 'int'})
data_set['생산시간'] = data_set['생산시간'].astype({'생산시간' : 'int'})
data_set.dtypes

## RandomForestClassifer 시도



In [ ]:
x = data_set.drop('생산시간',axis=1)
y = data_set['생산시간']

# x축에 생산시간칼럼을 제외한 데이터 값을 넣어준다
# y축에 생산시간컬럼만 넣어준다.

In [ ]:
x_tn, x_te, y_tn, y_te = train_test_split(x,y, test_size=0.2, random_state=0) # 학습시킬 데이터, 테스트용 데이터 설정


# 스케일링 하는 곳
std_scale = StandardScaler()
std_scale.fit(x_tn)
x_tn_std = std_scale.transform(x_tn)
x_te_std = std_scale.transform(x_te)

# 모델생성 randomforest로 만들어줌
clf_rf = RandomForestClassifier(max_depth=2, random_state=0)
clf_rf.fit(x_tn_std,y_tn)

In [ ]:
# 예측값 class에 들어가는 예측값을 넣는다.
pred_rf = clf_rf.predict(x_te_std)
print(pred_rf)

In [ ]:
# 정확도 검사
accuracy = accuracy_score(y_te, pred_rf)
print(accuracy)

In [ ]:
clf_rf.score(x_tn_std, y_tn)

## RandomForest Regressor 시도

In [ ]:
x = data_set.drop('생산시간',axis=1)
y = data_set['생산시간']

# x축에 생산시간칼럼을 제외한 데이터 값을 넣어준다
# y축에 생산시간컬럼만 넣어준다.
print(x.shape)
print(y.shape)

In [ ]:
x_tn, x_te, y_tn, y_te = train_test_split(x,y, test_size=0.2, random_state=0) # 학습시킬 데이터, 테스트용 데이터 설정

# 모델생성 randomforest로 만들어줌
model = RandomForestRegressor(criterion='mse', max_depth=20, random_state=0, min_samples_leaf=1, n_estimators=100)
model.fit(x_tn,y_tn)
y_pred=model.predict(x_tn)
print("Train Set Accuracy:",model.score(x_tn, y_tn).round(2))
print('Train Mean Absolute Error:', metrics.mean_absolute_error(y_tn, y_pred).round(2))  
print('Train Mean Squared Error:', metrics.mean_squared_error(y_tn, y_pred).round(2))  
print('Train Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_tn, y_pred)).round(2))  
print('Train Variance score: ', r2_score(y_tn,y_pred).round(2))

In [ ]:
# 중요인자 순서도를 구하는 코드 
importances = model.feature_importances_
std = np.std([model.feature_importances_ for tree in model.estimators_],axis=0)
indices = np.argsort(importances)[::-1]
feature_list = [x_tn.columns[indices[f]] for f in range(x_tn.shape[1])]
ff = np.array(feature_list)

plt.figure(figsize=(20,5))
plt.title("인자 중요도(Mean Decrease Accuracy)")
plt.bar(range(x_tn.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(x_tn.shape[1]), ff[indices], rotation=90)
plt.xlim([-1, x_tn.shape[1]])
plt.show()
print("중요인자 중요도순서:")
for f in range(x_tn.shape[1]):
    print("# %d.: %s" % (f + 1,  ff[indices[f]]))

In [ ]:
plt.figure(figsize=(10,10))

plot_x = np.array(y_pred)
plot_y = np.array(y_tn.tolist())

ax = sns.regplot(x=plot_x, y=plot_y)

## 민솔님 지원

In [ ]:
# 데이터 설정
x=data_set['쿠킹스팀압력'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=8)
clf.fit(x_train_sc, y_tn)

# train 정확도
clf.score(x_tn, y_tn)

# test 정확도
clf.score(x_te, y_te)

In [ ]:
# 데이터 설정
x=data_set['쿠킹스팀압력'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=5)
clf.fit(x_train_sc, y_tn)

# train 정확도
clf.score(x_tn, y_tn)

# test 정확도
clf.score(x_te, y_te)

In [ ]:
# 데이터 설정
x=data_set['쿠킹스팀압력'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=5)
clf.fit(x_train_sc, y_tn)

# train 정확도
print(clf.score(x_tn, y_tn))

# test 정확도
print(clf.score(x_te, y_te))

In [ ]:
# 데이터 설정
x=data_set['쿠킹스팀압력'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=10)
clf.fit(x_train_sc, y_tn)

# train 정확도
print(clf.score(x_tn, y_tn))

# test 정확도
print(clf.score(x_te, y_te))

## GradientBoosing

In [ ]:
# 데이터 설정
x=data_set['쿠킹스팀압력'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(max_depth=5)
clf.fit(x_train_sc, y_tn)

# train 정확도
clf.score(x_tn, y_tn)

# test 정확도
clf.score(x_te, y_te)

In [ ]:
# 데이터 설정
x=data_set['쿠킹스팀압력'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(max_depth=8)
clf.fit(x_train_sc, y_tn)

# train 정확도
clf.score(x_tn, y_tn)
print(clf.score(x_tn, y_tn))

# test 정확도
clf.score(x_te, y_te)
print(clf.score(x_te, y_te))

In [ ]:
# 데이터 설정
x=data_set['충전실온도'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(max_depth=8)
model.fit(x_train_sc, y_tn)

# train 정확도
model.score(x_tn, y_tn)
print(model.score(x_tn, y_tn))

# test 정확도
model.score(x_te, y_te)
print(model.score(x_te, y_te))

#TEST

In [ ]:
# 데이터 설정
x=data_set['쿠킹스팀압력'] # input
y=data_set['생산시간'] # target


# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tn)
x_train_sc = scaler.transform(x_tn)
x_test_sc = scaler.transform(x_te)

# 학습
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=8)
clf.fit(x_train_sc, y_tn)

# train 정확도
clf.score(x_tn, y_tn)
print(model.score(x_tn, y_tn))

# test 정확도
clf.score(x_te, y_te)
print(model.score(x_te, y_te))